#### Imports for running Data Scraping notebook:

In [1]:
import pandas as pd
from pmaw import PushshiftAPI
import praw
from datetime import datetime

### PRAW Reddit API
Using the PRAW API to collect the desired information about Reddit posts in the r/climatechange subreddit.

In [8]:
# Instantiating API for PRAW:
reddit = praw.Reddit(client_id='xSiGiGwvCH2RY95RjB2WKQ', client_secret='NJdd_bWhk4HpdnmTIEMjMlj6sIY5Ow', 
                     user_agent='project 3')

In [9]:
# Setting up scrape to collect only the desired features:
posts = []
cc_subreddit = reddit.subreddit('climatechange')
for post in cc_subreddit.new(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, 
                  post.num_comments, post.selftext, post.created])

posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts.head()

,title,score,id,subreddit,url,num_comments,body,created
0,Increasingly large and intense forest fires hi...,40,11ot7du,climatechange,https://insideclimatenews.org/news/06032023/wi...,6,,1.678564e+09
1,Large Contribution of Ozone-Depleting Substanc...,4,11ortk9,climatechange,https://agupubs.onlinelibrary.wiley.com/doi/10...,2,,1.678561e+09
2,Accurate American carbon footprint,1,11og4mw,climatechange,https://www.reddit.com/r/climatechange/comment...,10,It’s hard to find an accurate number for a typ...,1.678528e+09
3,Arctic ice melting is an issue!,40,11o85ag,climatechange,https://www.reddit.com/r/climatechange/comment...,21,I read about cloud seeding and it is a great w...,1.678501e+09
4,Teacher looking to transition to climate change,27,11nklx2,climatechange,https://www.reddit.com/r/climatechange/comment...,16,"Hi all, \n\nAs the title says, I am a fully qu...",1.678441e+09


In [11]:
# Converting the epoch time coding to a new, readable 'date' column:
posts['date'] = pd.to_datetime(posts['created'],unit='s')
posts.head()

,title,score,id,subreddit,url,num_comments,body,created,date
0,Increasingly large and intense forest fires hi...,40,11ot7du,climatechange,https://insideclimatenews.org/news/06032023/wi...,6,,1.678564e+09,2023-03-11 19:47:53
1,Large Contribution of Ozone-Depleting Substanc...,4,11ortk9,climatechange,https://agupubs.onlinelibrary.wiley.com/doi/10...,2,,1.678561e+09,2023-03-11 18:52:20
2,Accurate American carbon footprint,1,11og4mw,climatechange,https://www.reddit.com/r/climatechange/comment...,10,It’s hard to find an accurate number for a typ...,1.678528e+09,2023-03-11 09:47:01
3,Arctic ice melting is an issue!,40,11o85ag,climatechange,https://www.reddit.com/r/climatechange/comment...,21,I read about cloud seeding and it is a great w...,1.678501e+09,2023-03-11 02:18:55
4,Teacher looking to transition to climate change,27,11nklx2,climatechange,https://www.reddit.com/r/climatechange/comment...,16,"Hi all, \n\nAs the title says, I am a fully qu...",1.678441e+09,2023-03-10 09:41:30


In [12]:
# Confirming we have desired columns:
posts.columns

Index(['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body',
       'created', 'date'],
      dtype='object')

In [14]:
# Checking to see how many posts were collected:
len(posts)

957

### PMAW - PushshiftAPI
Since the limitations with PRAW were such that I could only collect ~1k posts, I will use PMAW to supplement so that I have an adequate amount of data to use.

In [2]:
# Instantiate our API for PMAW:
api = PushshiftAPI(num_workers=5)

In [3]:
# Make the request for our posts in the desired subreddit
# Using a custom filter for only posts with at least 1 comment:
def fxn(item):
  return item['num_comments'] > 0
psaw_posts = api.search_submissions(filter_fn=fxn, subreddit="climatechange", limit=30000)

In [4]:
# Confirming call was successful:
print(f'{len(psaw_posts)} posts retrieved')

0 posts retrieved


Update 3/11/23 - this process no longer works 6 months later due to changes with Pushshift.

In [116]:
# Reading in posts into a list to be able to convert:
post_list = [p for p in psaw_posts]

In [117]:
# convert submissions to dataframe
new_posts_df = pd.DataFrame(post_list)

In [118]:
# Looking at columns to see what I want to keep:
new_posts_df.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_is_blocked',
       'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post',
       'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_created_from_ads_ui', 'is_crosspostable', 'is_meta',
       'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable',
       'is_self', 'is_video', 'link_flair_background_color',
       'link_flair_richtext', 'link_flair_text_color', 'link_flair_type',
       'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts',
       'over_18', 'parent_whitelist_status', 'permalink', 'pinned',
       'post_hint', 'preview', 'pwls', 'retrieved_on', 'score', 'selftext',
       'send_replies', 'spoiler', 'stickied', 'subreddit', 'subreddit_id',
       'subreddit_subscribers', 'subreddit_type', 'thumbnail',
   

In [1]:
# Creating dataframe with desired columns, and those that match the first scrape from PRAW:
better_df = new_posts_df[['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'selftext',
       'created_utc']].copy()

better_df.head()

NameError: name 'new_posts_df' is not defined

In [121]:
better_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9188 entries, 0 to 9187
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         9188 non-null   object
 1   score         9188 non-null   int64 
 2   id            9188 non-null   object
 3   subreddit     9188 non-null   object
 4   url           9188 non-null   object
 5   num_comments  9188 non-null   int64 
 6   selftext      9173 non-null   object
 7   created_utc   9188 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 574.4+ KB


In [122]:
# Converting UNIX/epoch time to readable:
better_df['date'] = pd.to_datetime(better_df['created_utc'],unit='s')

In [127]:
better_df.head()

,title,score,id,subreddit,url,num_comments,selftext,created_utc,date
0,The 'doomed' UK village to be abandoned to the...,1,qsj557,climatechange,https://www.nzherald.co.nz/world/the-doomed-uk...,9,,1636746293,2021-11-12 19:44:53
1,Nearly-fatal heat already affects people in ‘i...,1,qsg7f3,climatechange,https://idp.nature.com/authorize?response_type...,3,,1636738068,2021-11-12 17:27:48
2,🔮 93% of chance the atmospheric nitrous oxide ...,1,qscf7q,climatechange,https://www.reddit.com/r/climatechange/comment...,6,[https://futuur.com/q/142710/will-atmospheric...,1636727544,2021-11-12 14:32:24
3,We are nature,1,qs35pn,climatechange,https://i.redd.it/r1waqyb6c3z71.jpg,1,,1636690680,2021-11-12 04:18:00
4,The Climate Is Changing: But That Is Just The ...,1,qryv4m,climatechange,https://www.reddit.com/r/climatechange/comment...,1,"Scriptural Foundation: Matthew 24:3-7, See als...",1636676231,2021-11-12 00:17:11


In [125]:
better_df.shape

(9188, 9)

#### Merging PRAW and PMAW dataframes:

In [130]:
# Changing column names to match each other:
better_df.rename(columns = {'selftext':'body', 'created_utc':'created'}, inplace = True)

In [131]:
print(posts.columns)
print(better_df.columns)

Index(['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body',
       'created', 'date'],
      dtype='object')
Index(['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body',
       'created', 'date'],
      dtype='object')


In [135]:
df_merged = pd.concat([posts, better_df], axis=0)
df_merged.shape

(10161, 9)

In [136]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10161 entries, 0 to 9187
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         10161 non-null  object        
 1   score         10161 non-null  int64         
 2   id            10161 non-null  object        
 3   subreddit     10161 non-null  object        
 4   url           10161 non-null  object        
 5   num_comments  10161 non-null  int64         
 6   body          10146 non-null  object        
 7   created       10161 non-null  float64       
 8   date          10161 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 793.8+ KB


### Saving to CSV

In [137]:
df_merged.to_csv('cc_posts.csv', header=True, index=False, columns=list(better_df.axes[1]))